# 🤖 CyanoManus 




### import  libraries

In [ ]:
# for better dispay 
from IPython.display import display, Markdown, Latex, JSON, Image, YouTubeVideo, Markdown, HTML, SVG, JSON, Pretty
from rich import print as rprint
from rich import print_json

# openai & openai agents 
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    trace,
    set_default_openai_api,
    set_default_openai_client,
    set_trace_processors
)

# langsmith tracing
from langsmith.wrappers import OpenAIAgentsTracingProcessor

# .env load 
from dotenv import load_dotenv, find_dotenv
import os 

# asyncio 
import asyncio
import nest_asyncio

In [ ]:
import sys 
from pathlib import Path

cwd_dir = Path(os.getcwd())
root_dir = cwd_dir.parent
sys.path.append(str(root_dir))
    
rprint(root_dir)

### configure

In [ ]:
#
_ = load_dotenv(find_dotenv())

#
external_client = AsyncOpenAI(
    api_key=os.getenv("BASIC_LLM_API_KEY"),
    base_url=os.getenv("BASIC_LLM_BASE_URL"),
)

set_default_openai_client(external_client, use_for_tracing=True)
set_default_openai_api("chat_completions") 

#
set_trace_processors([OpenAIAgentsTracingProcessor()])

# 
nest_asyncio.apply()
loop = asyncio.get_event_loop()

### build start agent

In [ ]:
from src.function_tools.search import prompt_with_tavily_tools_instructions
from src.function_tools.search import tavily_search, tavily_extract, get_weather

from src.utils.cyano_hooks import CyanoAgentHooks

instructions_str = "你是一个 AI 智能体，擅长使用各种网络搜索工具完成用户的请求。"


starting_agent = Agent(
    name="research_orch_agent",
    instructions=prompt_with_tavily_tools_instructions(instructions_str),
    tools=[tavily_search, tavily_extract, get_weather],
    hooks=CyanoAgentHooks(title="搜索智能体")
)

### agent invoke

In [ ]:
async def main():
    input_str = "能不能告诉我杭州今天的天气状况?"
    result = await Runner.run(
        starting_agent=starting_agent, 
        input=input_str
    ) 
    rprint(result.final_output)

In [ ]:
await main()

### multi-round chat 

In [ ]:
async def chat(starting_agent : Agent):
    with trace("main_nb-chat_with_agent"):
        messages = []
        
        while True:
            user_input = input("User: ")
            if user_input.lower() in ["exit", "bye", "quit"]:
                break
            # invoke agent
            messages.append({"role": "user", "content": user_input})
            result = await Runner.run(
                starting_agent=starting_agent,
                input=messages
            )
            # display result
            display(Markdown(result.final_output))
            # over write messages
            messages = result.to_input_list()

In [ ]:
await chat(
    starting_agent=starting_agent
)

### DEBUG 

In [ ]:
from src.function_tools.search import get_weather

In [ ]:
weather_data = await get_weather(loc="hangzhou")

In [ ]:
weather_data